<a href="https://colab.research.google.com/github/gaurav8341/PATE-Analysis/blob/master/PATE-Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install syft

In [2]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import Subset
import numpy as np
from syft.frameworks.torch.differential_privacy import pate

W0730 19:42:55.422148 140472506627968 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0730 19:42:55.444672 140472506627968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



# Loading dataset


In [0]:


# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)#need to keep differentially private
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)#no need to keepdifferentially private


# Creating the dataset for each teacher model


In [0]:
num_teacher = 100 # need to check if number of teacher affects the epsilon
num_examples = len(train_data)//num_teacher # the number of datapoints alloacated to each teacher
batch_size = 64 # numberr of samples in each batch
num_workers = 0 # number of subprocesses for data loading ????

teacher_loader = []
for i in range(num_teacher):
  indices = list(range(i * num_examples, (i+1) * num_examples))
  data = Subset(train_data, indices)
  loader = torch.utils.data.DataLoader(data, batch_size = batch_size, num_workers = num_workers)
  teacher_loader.append(loader)
  


# Creating Data set for Student model.

In [0]:
train_size = int(len(test_data) * 0.05) # for training size
test_size = int(len(test_data) * 0.05) # for testing size

student_train_data = Subset(test_data, list(range(train_size))) # training data
student_test_data = Subset(test_data, list(range(train_size, train_size + test_size))) # testing data

student_train_loader = torch.utils.data.DataLoader(student_train_data, batch_size = batch_size, num_workers = num_workers) #training data loader
student_test_loader = torch.utils.data.DataLoader(student_test_data, batch_size = batch_size, num_workers = num_workers) #testing data loader

# Defining Neural Network Model

In [0]:
from torch import nn, optim
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(784 , 256)
        self.fc2 = nn.Linear(256 , 128)
        self.fc3 = nn.Linear(128 ,64)
        self.fc4 = nn.Linear(64 , 10)
        self.dropout = nn.Dropout(p = 0.2)
        
    def forward(self , x):
        x = x.view(x.shape[0], -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        
        x = F.log_softmax(self.fc4(x))
        
        return x

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# An All Inclusive Function to Train Models 
This function can be used to train both teacher model and student model.

In [0]:
def train(model, trainloader, criterion, optimizer, epochs = 10, testloader = None):
  model.to(device)
  running_loss = 0
  steps = 0
  
  for e in range(epochs):
    model.train() # initializing model in training mode
   
    for images, labels in trainloader:
      images, labels = images.to(device) , labels.to(device)
      images = images.view( images.shape[0],-1)

      steps += 1
      
      # erase perv grad
      optimizer.zero_grad()
      
      # forward the image
      pred = model.forward(images)
      
      #calculate loss
      loss = criterion(pred, labels)
      
      #backpropagation
      loss.backward()
      
      #change weights
      optimizer.step()
      
      #save the loss 
      running_loss += loss.item()
      
      if(steps % 25 == 0 and testloader != None):
        test_loss = 0
        accuracy = 0
        model.eval()#set in evaluation mode
        with model.no_grad():
          for images, labels in testloader:
            images, labels = images.to(device) , labels.to(device)
            images = images.view( images.shape[0],-1)
            log_ps = model(images)
            test_loss += criterion(log_ps, labels).item()
            
            #accuracy
            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor))
        model.train()
        
        print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
                      "Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
                      "Test Accuracy: {:.3f}".format(accuracy/len(test_loader)))
        running_loss = 0
      
    
    

# Function to Predict outputs from Models.

In [0]:
def predict(model, dataloader):
  outputs = torch.zeros(0 , dtype = torch.long).to(device)
  model.to(device)
  model.eval()
  for images, labels in dataloader:
    images = images.view( images.shape[0],-1)
    images, labels = images.to(device), labels.to(device)
    output = model.forward(images)
    ps = torch.argmax(torch.exp(output), dim=1)
    outputs = torch.cat((outputs, ps))
    
  return outputs

# Train all Teacher Model.

In [10]:
teacher_models = []

for i in range(num_teacher):
  print("Training Teacher #", i+1)
  model = Net()
  criterion = nn.NLLLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  train(model, teacher_loader[i], criterion, optimizer)
  teacher_models.append(model)

Training Teacher # 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Training Teacher # 2
Training Teacher # 3
Training Teacher # 4
Training Teacher # 5
Training Teacher # 6
Training Teacher # 7
Training Teacher # 8
Training Teacher # 9
Training Teacher # 10
Training Teacher # 11
Training Teacher # 12
Training Teacher # 13
Training Teacher # 14
Training Teacher # 15
Training Teacher # 16
Training Teacher # 17
Training Teacher # 18
Training Teacher # 19
Training Teacher # 20
Training Teacher # 21
Training Teacher # 22
Training Teacher # 23
Training Teacher # 24
Training Teacher # 25
Training Teacher # 26
Training Teacher # 27
Training Teacher # 28
Training Teacher # 29
Training Teacher # 30
Training Teacher # 31
Training Teacher # 32
Training Teacher # 33
Training Teacher # 34
Training Teacher # 35
Training Teacher # 36
Training Teacher # 37
Training Teacher # 38
Training Teacher # 39
Training Teacher # 40
Training Teacher # 41
Training Teacher # 42
Training Teacher # 43
Training Teacher # 44
Training Teacher # 45
Training Teacher # 46
Training Teacher #

# Aggregate Teacher

In [11]:
epsilon = 0.2

preds = torch.torch.zeros((len(teacher_models), train_size) , dtype = torch.long)

for i, model in enumerate(teacher_models):
  results = predict(model , student_train_loader)
  preds[i] = results

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
labels = np.array([]).astype(int)
for image_preds in np.transpose(preds):
  labels_counts = np.bincount(image_preds , minlength = 10)
  beta = 1/epsilon
  
  for i in range(len(labels_counts)):
    labels_counts[i] += np.random.laplace(0, beta, 1)
    
  new_labels = np.argmax(labels_counts)
  labels = np.append(labels , new_labels)
  

In [13]:
PATE_labels = labels # Aggregate Teacher Labels   
true_labels = test_data.targets[:train_size] # True Labels
PATE_preds = preds # Labels Obtained by Teachers

print(preds.shape)
print(labels.shape)
print("All Teacher Predicted Labels\n", preds)
print("Aggregate Teacher Labels\n", torch.tensor(PATE_labels))
print("True Labels\n", true_labels)

torch.Size([100, 500])
(500,)
All Teacher Predicted Labels
 tensor([[7, 2, 1,  ..., 9, 0, 6],
        [7, 5, 1,  ..., 9, 9, 6],
        [7, 6, 1,  ..., 9, 5, 6],
        ...,
        [7, 2, 1,  ..., 9, 0, 6],
        [7, 8, 1,  ..., 9, 9, 6],
        [7, 3, 1,  ..., 4, 9, 5]])
Aggregate Teacher Labels
 tensor([7, 2, 1, 0, 4, 1, 4, 9, 6, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 6, 7, 2, 7, 1, 3, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 2, 0, 4, 1, 9, 5, 7, 8, 4, 2, 7, 4, 2, 4, 3, 0, 7, 0,
        2, 8, 1, 7, 3, 7, 9, 7, 9, 6, 2, 7, 8, 4, 7, 5, 6, 1, 3, 6, 4, 3, 1, 4,
        1, 1, 6, 9, 6, 0, 5, 4, 4, 9, 2, 1, 9, 4, 8, 1, 3, 9, 7, 4, 4, 4, 9, 2,
        5, 4, 7, 6, 9, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1,
        7, 1, 8, 2, 0, 4, 4, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5,
        1, 4, 4, 7, 3, 3, 2, 1, 1, 8, 1, 8, 1, 8, 5, 0, 8, 4, 2, 3, 0, 1, 1, 1,
        0, 9, 0, 1, 1, 6, 4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 

In [0]:
dat_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds = PATE_preds, indices = PATE_labels, noise_eps = epsilon, delta = 1e-5, moments = 10)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)